In [ ]:
%matplotlib inline


Particle Tracker
================

An example of PlasmaPy's particle tracker class.


In [ ]:
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np

from plasmapy.formulary import ExB_drift, gyrofrequency
from plasmapy.particles import Particle
from plasmapy.plasma.grids import CartesianGrid
from plasmapy.simulation.particle_tracker import (
    IntervalSaveRoutine,
    ParticleTracker,
    TimeElapsedTerminationCondition,
)

Initialize a :class:`~plasmapy.plasma.grids.CartesianGrid` object. This will be the source of electric and magnetic
fields for our particles to move in.



In [ ]:
grid_length = 10
grid = CartesianGrid(-1 * u.m, 1 * u.m, num=grid_length)

Initialize the fields. We'll take B in the x direction
and E in the y direction, which gets us an E cross B drift
in the negative z direction.



In [ ]:
Bx_fill = 4 * u.T
Bx = np.full(grid.shape, Bx_fill.value) * u.T

Ey_fill = 2 * u.V / u.m
Ey = np.full(grid.shape, Ey_fill.value) * u.V / u.m

grid.add_quantities(B_x=Bx, E_y=Ey)
ExB_drift(np.asarray([0, Ey_fill.value, 0]) * u.V / u.m, np.asarray([Bx_fill.value, 0, 0]) * u.T)

|ParticleTracker| takes arrays of particle positions and velocities of the shape [nparticles, 3], so these arrays represent one particle starting at the origin.

In [ ]:
x0 = [[0, 0, 0]] * u.m
v0 = [[1, 0, 0]] * u.m / u.s
particle = Particle("p+")

Initialize our stop condition and save routine. We can determine a relevant
duration for the experiment by calculating the gyroperiod for the particle.

In [ ]:
particle_gyroperiod = 1 / gyrofrequency(Bx_fill, particle).to(u.Hz, equivalencies=u.dimensionless_angles())

simulation_duration = 100 * particle_gyroperiod
save_interval = particle_gyroperiod / 10

termination_condition = TimeElapsedTerminationCondition(simulation_duration)
save_routine = IntervalSaveRoutine(save_interval)

Initialize the trajectory calculation.

In [ ]:
simulation = ParticleTracker(grid, save_routine=save_routine, termination_condition=termination_condition, verbose=False)

We still have to initialize the particle's velocity. We'll limit ourselves to
one in the x direction, parallel to the magnetic field B -
that way, it won't turn in the z direction.



In [ ]:
simulation.load_particles(x0, v0, particle)

Run the simulation.

In [ ]:
simulation.run()

We can take a look at the trajectory in the z direction:

In [ ]:
t, x, v = save_routine.results()
particle_trajectory = x[:, 0]
particle_position_z = particle_trajectory[:, 2]

plt.scatter(t, particle_position_z)

or plot the shape of the trajectory in 3D:

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

ax.plot(*particle_trajectory.T)
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")

As a test, we calculate the mean velocity in the z direction from the
velocity:

In [ ]:
v_mean = v[:, :, 2].mean()
print(
    f"The calculated drift velocity is {v_mean:.4f} to compare with the "
    f"expected E0/B0 = {-(Ey_fill/Bx_fill).value:.4f}"
)